# Preprocesamiento de datos PAC

Limpieza y preparación de datos para model de detección de anomalías.
Utiliza datos de los Beneficiarios de Ayuda de la Política Agrícola Común de 2024.

https://www.fega.gob.es/es/datos-abiertos/consulta-de-beneficiarios-pac/descarga-de-ficheros

## Preparar ambiente

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, types as T
import pyspark.pandas as ps
import unicodedata

/Users/lizziecanamar/Documents/MASTER/TFM/Implementations/ml-spark-use-case/.venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Iniciar Spark session
spark = SparkSession.builder.appName("PAC_Preprocesado").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/10 11:29:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/10 11:29:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
input_path = "data/Beneficiarios_municipio_ejercicio_financiero_2024.txt"
output_path = "data/pac_2024_clean/"

## Limpieza

In [4]:
# Leer datos
df_raw = spark.read.csv(input_path,sep=";", header=True, encoding="latin1", inferSchema=False)
df_raw.show(5)

+--------------------+-------------+---------+-------------+--------------------+------------+-------+-------+--------+------+------------+------------+-------------+
|        BENEFICIARIO|GRUPO_EMPRESA|PROVINCIA|    MUNICIPIO|              MEDIDA|OBJETIVO_ESP|FEC_INI|FEC_FIN|   FEAGA|FEADER|IMPORTECOFIN|FEADER_COFIN|IMPORTE_EUROS|
+--------------------+-------------+---------+-------------+--------------------+------------+-------+-------+--------+------+------------+------------+-------------+
|: HERENCIA YACENT...|         null|   Madrid|28982 - Parla|II.1   Régimen de...|        null|   null|   null|24217,98|     0|           0|           0|     24217,98|
|: HERENCIA YACENT...|         null|   Madrid|28982 - Parla|II.4   Pago para ...|        null|   null|   null|12650,71|     0|           0|           0|     12650,71|
|: HERENCIA YACENT...|         null|   Madrid|28982 - Parla|I.6   Ayuda a la ...|     OE2|OE6|   null|   null|16969,68|     0|           0|           0|     16969,68

In [5]:
# Duplicados
print("Antes de quitar duplicados: ",df_raw.count())
df = df_raw.dropDuplicates()
print("Después de quitar duplicados: ",df.count())

Antes de quitar duplicados:  2217849


Después de quitar duplicados:  2217834


In [6]:
# Porcentaje de nulos (para análisis)
nulls = df.select([ (F.count(F.when(F.col(c).isNull(), c)) / F.count("*")).alias(c) for c in df.columns ])
nulls.show(truncate=False)

+------------+-----------------+---------+---------+------+--------------------+------------------+------------------+---------------------+------+------------+------------+-------------+
|BENEFICIARIO|GRUPO_EMPRESA    |PROVINCIA|MUNICIPIO|MEDIDA|OBJETIVO_ESP        |FEC_INI           |FEC_FIN           |FEAGA                |FEADER|IMPORTECOFIN|FEADER_COFIN|IMPORTE_EUROS|
+------------+-----------------+---------+---------+------+--------------------+------------------+------------------+---------------------+------+------------+------------+-------------+
|0.0         |0.998159465496516|0.0      |0.0      |0.0   |0.057901989057792425|0.9971219667477368|0.9981630726195018|4.5089037321999755E-7|0.0   |0.0         |0.0         |0.0          |
+------------+-----------------+---------+---------+------+--------------------+------------------+------------------+---------------------+------+------------+------------+-------------+



In [7]:
# Mostrar columnas donde los nulos > 65%
columns = [col for col in nulls.columns if col != "summary"]

unpivoted_df = nulls.selectExpr("stack(" + str(len(columns)) + ", " +
                             ", ".join([f"'{col}', `{col}`" for col in columns]) +
                             ") as (Column, Value)")

filtered_df = unpivoted_df.filter(F.col("Value") > 0.65)
filtered_df.show(truncate=False)

25/09/10 11:30:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+------------------+
|Column       |Value             |
+-------------+------------------+
|GRUPO_EMPRESA|0.998159465496516 |
|FEC_INI      |0.9971219667477368|
|FEC_FIN      |0.9981630726195018|
+-------------+------------------+



In [8]:
# Excluir columnas con >65% nulos
df_clean = df.drop("GRUPO_EMPRESA", "FEC_INI", "FEC_FIN")

### Ajuste de esquema/tipos de dato

In [9]:
# Convertir decimales con coma a punto
num_cols = ["FEAGA","FEADER","IMPORTECOFIN","FEADER_COFIN","IMPORTE_EUROS"]

for c in num_cols:
    df_clean = df_clean.withColumn(c, F.regexp_replace(c, ",", ".").cast(T.DoubleType()))

df_clean = ( df_clean
            # Beneficiario: quitar prefijo ": " y trim
            .withColumn("BENEFICIARIO", F.trim(F.regexp_replace("BENEFICIARIO", "^: ?", "")))
            # Separar código y nombre de los municipios
            .withColumn("MUNICIPIO_COD", F.trim(F.split("MUNICIPIO", "-").getItem(0)))
            .withColumn("MUNICIPIO_NOMBRE", F.trim(F.split("MUNICIPIO", "-").getItem(1)))
            # Convertir nulos en string vacío
            .withColumn("OBJETIVO_ESP", F.coalesce(F.col("OBJETIVO_ESP"), F.lit("")))
            ).drop("MUNICIPIO")


### Agregar flag de recuperaciones

In [10]:
# Flag  por importe negativo o MEDIDA que contenga 'Recuperaciones'
df_clean = (df_clean
           .withColumn("IS_RECUP", (F.col("IMPORTE_EUROS") < 0).cast("int"))
           .withColumn("IS_RECUP_MEDIDA", F.locate("Recuperaciones", F.col("MEDIDA")).cast("int"))
           .withColumn("IS_RECUP_ANY", ((F.col("IS_RECUP") == 1) | (F.col("IS_RECUP_MEDIDA") > 0)).cast("int")))

### Limpieza nombres de provincias

In [11]:
bilingual_map = {
    "València/Valencia": "Valencia",
    "Alacant/Alicante": "Alicante",
    "Castelló/Castellón": "Castellon",  
}

# UDF para normalizar a ASCII (quitar acentos)
def strip_accents(s):
    if s is None:
        return None
    s = s.strip()
    s = bilingual_map.get(s, s)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = " ".join(s.split())
    s = s.replace(" ", "-")
    return s

strip_accents_udf = F.udf(strip_accents, T.StringType())

df_out = df_clean.withColumn("PROVINCIA_SAFE", strip_accents_udf(F.col("PROVINCIA")))

In [12]:
# Verificar distribución tras normalizar
#df_out.groupBy("PROVINCIA", "PROVINCIA_SAFE").count().orderBy(F.desc("count")).show(100, truncate=False)

In [13]:
# Duplicados
print("Antes de quitar duplicados: ",df_clean.count())
df_out = df_out.dropDuplicates()
print("Después de quitar duplicados: ",df_out.count())

Antes de quitar duplicados:  2217834


Después de quitar duplicados:  2217817


## Guardar parquet limpio

In [14]:
(df_out
    .write.mode("overwrite")
    .partitionBy("PROVINCIA_SAFE")
    .option("compression", "snappy")
    .parquet(output_path)
)

25/09/10 11:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/09/10 11:31:46 WARN MemoryManager: Total allocation exceeds 95.00% 

In [15]:
# Clear the cache in Spark
spark.catalog.clearCache()

# Stop the Spark session
spark.stop()